In [1]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import timeit
from decimal import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifergutman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import language_tool_python

In [3]:
tool = language_tool_python.LanguageTool('en-US')

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jennifergutman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# !pip install gingerit

In [6]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i.lower())
words_df = temp[:5000] 

In [7]:
tokens_tag = pos_tag(words_df)
dic_tag = {}
for tup in tokens_tag:
  if tup[1] not in dic_tag.keys():
    dic_tag[tup[1]] = [tup[0]]
  else:
    temp = dic_tag[tup[1]] + [tup[0]]
    dic_tag[tup[1]] = temp
# print(dic_tag.keys())
z = list(dic_tag.values())
words_df2 = []
for i in range(len(z)):
  words_df2.extend(z[i])
# print(words_df2)
words_df = words_df2
words_df.remove('o')
words_df.remove('n')

In [8]:
lines = []
for i in range(3, len(words_df)-3):
    lines.append(words_df[i-3]+' '+' '+words_df[i-2]+' '+' '+words_df[i-1]+' '+' '+words_df[i]+' '+' '+words_df[i+1]+' '+' '+words_df[i+2]+' '+' '+words_df[i+3])

# sklearn countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Convert a collection of text documents to a matrix of token counts
cv = CountVectorizer(ngram_range=(1,1), stop_words = stopwords.words('english'))
# matrix of token counts
X = cv.fit_transform(lines)
Xc = (X.T * X) # matrix manipulation
Xc.setdiag(0) # set the diagonals to be zeroes as it's pointless to be 1

names = cv.get_feature_names() # This are the entity names (i.e. keywords)
cooccur_df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
total = cooccur_df.to_numpy().sum()
cooccur_df = cooccur_df.div(total)
cooccur_df.to_csv('co_occur_mat.csv', sep = ',')

In [13]:
class poem_generation:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    def select_random_words(self,words_df):
        words_array1 = []
        while len(words_array1) <= 15:
            random_number = random.randint(0, len(words_df)-1)
            if words_df[random_number] != 0:
                words_array1.append(words_df[random_number])
        # print(len(words_array1))

        return words_array1

    def create_poems(self,arr):
        poem = [0 for _ in range(4)]
        for i in range(1,5):
            temp =  arr[4*(i-1):4*i]
            poem[i-1] = temp
            # print("HERE:",temp)
            # print(type(temp))
            poem[i-1]=' '.join(poem[i-1])
            # print(poem[i-1])
        # print("=========================================================")
        return poem

    def store_syllables(self,arr):
        a = [0 for _ in range(4)]
        temp = 0
        j = 0
        for words,i in zip(arr,range(0,16)):
            temp = temp + syllables.estimate(words)
            if i in [3,7,11,15]:
                a[j] = temp
                # print(a[j])
                temp = 0
                j += 1
        return a

    def Grammar_checker(self,text):
        error = 0
        poem_error = []
        # tool = language_tool_python.LanguageTool('en-US')
        for lines in text:
            matches = tool.check(lines)
            if len(matches)>0:
                error = error + matches[0].errorLength
        poem_error.append(error)
        return poem_error

    def crossover(self,p1,p2,cross):
        part1 = []
        part2 = []
        part3 = []
        child1 = []
        child2 = []
        j = 0
        k = 0
        random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
        random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
        for i in random_number1:
            part1.append(p1[j:i])
            j = i
        for i in random_number2:
            part2.append(p2[k:i])
            k = i
        part1.append(p1[j:])
        part2.append(p2[k:])
        part2.extend(part1)
        random.shuffle(part2)
        for i in range(len(part2)):
            part3.extend(part2[i])
        for i in range(int(len(part3)/2)):
            child1.append(part3[i])
        for i in range(int(len(part3)/2),int(len(part3))):
            child2.append(part3[i])
        # if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
        #     child1, child2,child1_poem,child2_poem = self.crossover(child1,child2,cross)
        child1_poem = self.create_poems(child1)
        child2_poem = self.create_poems(child2)
        return child1,child2,child1_poem,child2_poem
    
    def coherence_checker(self, text):
        clean_text = ''
        for i in text:
            clean_text = clean_text + ' ' + i
        
        text_tokens = word_tokenize(clean_text)
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]
        
        coherence_val = 0
        for i in range(len(tokens_without_sw) - 1):
            for j in range(i+1, len(tokens_without_sw)):
                coherence_val += cooccur_df[tokens_without_sw[i]][tokens_without_sw[j]]
            
        coherence_val /= (len(tokens_without_sw)-1)
        #coherence_val
        return [coherence_val]

    def tournament(self,Population,m,best):
        from_best = len(best)
        # print("best:",best)
        except_best = m - from_best
        # df = pd.DataFrame()
        #selected_polulation = pd.DataFrame(columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","word_array"])
        ######################### when no syllables rule
        # selected_polulation = Population.iloc[:nn]
        # random_number = random.sample(list(Population.iloc[nn:].index),p)
        # Population.sort_values(by=['GrammarError'], inplace=True)
        # Population2 = Population[~Population.isin(selected_polulation).all(1)]
        ######################### when syllables rule
        if len(best) >0:
            selected_polulation = []
            words_array_selected = []
            for b in best:
                c = Population.iloc[b].values[:-1]
                d = Population.iloc[b].values[-1]
                selected_polulation.append(c)
                words_array_selected.append(d)
            words_array_selected = pd.DataFrame(dict(zip(["words_array"],[words_array_selected])))
            
############# This line was changed with addition of fitness column 
            #selected_polulation = pd.DataFrame(selected_polulation,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
            selected_polulation = pd.DataFrame(selected_polulation,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence","Gram Rank","Coh Rank","Syll_Bonus","Fitness"])
            
            selected_polulation = pd.concat([selected_polulation,words_array_selected],axis=1)
            # print("poems selected by user")
            # display(selected_polulation)
            Population2 = Population[~Population.isin(selected_polulation).all(1)]
            Population2 = Population2.sort_values(by=['GrammarError'])
            selected_polulation2 = Population2[(Population2["Syllables1"] == "5")&(Population2["Syllables2"] == "6")&(Population2["Syllables3"] == "6")&(Population2["Syllables4"] == "5")]
            # selected_polulation2 = Population2[(Population2["Syllables1"] >= "3")]
            # print("grammar sorted poems by syll")
            # display(selected_polulation2)
            from_sy = len(selected_polulation2.index)
            if from_sy >= int(except_best/2):
                from_sy = int(except_best/2)
            else:
                from_sy = from_sy
            from_random = except_best - from_sy
            selected_polulation2 = selected_polulation2.iloc[:from_sy]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            display(selected_polulation)
            display(Population2)
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            Population2 = Population2.reset_index(drop=True)
            selected_polulation = pd.concat([selected_polulation,Population2.iloc[:from_random]],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected
        else:
            Population2 = Population.sort_values(by=['GrammarError'])
            random = int(except_best/3)
            grammar = except_best - random
            selected_polulation = Population2[:grammar]
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            selected_polulation2 = Population2[:random]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected



    def create_set_of_poems(self,n):
            poem_population = []
            syllables_count = []
            grammar_error = []
            words_array_population = []
            coherence_arr = []
            for i in range(n):
                words_array = self.select_random_words(words_df)
                poem = self.create_poems(words_array)
                sum_syllables = self.store_syllables(words_array)
                poem_error = self.Grammar_checker(poem)
                coherence = self.coherence_checker(poem)
                poem_population.append(poem)
                syllables_count.append(sum_syllables)
                grammar_error.append(poem_error)
                coherence_arr.append(coherence)
                words_array_population.append(words_array)
            words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
            Final = np.hstack((poem_population,syllables_count,grammar_error,coherence_arr))
            Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
            
############# Calculate Fitness of Poems
            max_rank = len(Population_data["Line1"])
            Population_data["Gram_Rank"] = Population_data["GrammarError"].rank(method = 'min', ascending = False)
            Population_data["Coherence"] = Population_data["Coherence"].apply(lambda x: [float(el) for el in x.strip("[]").split(",")])
            Population_data["Coh_Rank"] = Population_data["Coherence"].rank(method = 'min')
            Population_data["Syll_Bonus"] = Population_data.apply(lambda row: 1 if ((row.Syllables1 == 5)&(row.Syllables2 == 6)&(row.Syllables3 == 6)&(row.Syllables4 == 5)) else 0, axis = 1)
            Population_data["Fitness"] = Population_data.apply(lambda row: 0.4*row.Gram_Rank/max_rank + 0.4*row.Coh_Rank/max_rank + 0.2*row.Syll_Bonus, axis = 1)
###########            
            Pop_data = pd.concat([Population_data,words_array2],axis=1)
            # print()
            # print("PARENTS")
            # display(Pop_data)

            return Pop_data

    def child_poems(self,words_array,cross):
        child_population = []
        syllables_count = []
        grammer_error = []
        words_array_child = []
        coherence_arr = []
        # print("words_array",words_array)
        # print(type(words_array))
        if len(words_array)%2 == 2:
            for i in range(0,len(words_array)-1):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                coherence1 = self.coherence_checker(child1_poem)
                coherence2 = self.coherence_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                coherence_arr.append(coherence1)
                coherence_arr.append(coherence2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        else:
            for i in range(0,len(words_array)-1,2):
                # print(words_array[i],words_array[i+1])
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                coherence1 = self.coherence_checker(child1_poem)
                coherence2 = self.coherence_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                coherence_arr.append(coherence1)
                coherence_arr.append(coherence2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
        Final1 = np.hstack((child_population,syllables_count,grammer_error,coherence_arr))
        child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
        
######### Calculate Fitness of Child Poems
        max_rank = len(child_data["Line1"])
        child_data["Gram_Rank"] = child_data["GrammarError"].rank(method = 'min', ascending = False)
        child_data["Coherence"] = child_data["Coherence"].apply(lambda x: [float(el) for el in x.strip("[]").split(",")])
        child_data["Coh_Rank"] = child_data["Coherence"].rank(method = 'min')
        child_data["Syll_Bonus"] = child_data.apply(lambda row: 1 if ((row.Syllables1 == 5)&(row.Syllables2 == 6)&(row.Syllables3 == 6)&(row.Syllables4 == 5)) else 0, axis = 1)
        child_data["Fitness"] = child_data.apply(lambda row: 0.4*row.Gram_Rank/max_rank + 0.4*row.Coh_Rank/max_rank + 0.2*row.Syll_Bonus, axis = 1)
########        
        child_data = pd.concat([child_data,words_array_child],axis=1)
        child_data = child_data.reset_index(drop=True)
        print()
        # print("CHILD")
        # display(child_data)
        return child_data

    def selection_from_child_parents(self,x,child_data,selected_Pop_data):
        reduce = (len(list(child_data.index)) + len(list(selected_Pop_data.index))) - x
        n = int(reduce/3)
        p = reduce - n
        if reduce > len(list(child_data.index)) + len(list(selected_Pop_data.index)):
            print("ERROR")
        else:
            if n > len(child_data):
                n = len(child_data)
                p = reduce - p
            if n == 0:
                n = 1
                p = 1
        d = pd.concat([selected_Pop_data,child_data],axis=0)
        d = d.reset_index(drop=True)
        # print("p:",p)
        # print("n:",n)
        random_number1 = random.sample(list(selected_Pop_data.index),p)
        random_number2 = random.sample(list(range(len(list(selected_Pop_data.index)),len(list(d.index)))),n)
        random_number1.extend(random_number2)
        # print(random_number1)
        for i in random_number1:
          d.drop(i,axis=0,inplace = True)
        d = d.reset_index(drop=True)
        # print("NEW POPULATION")
        # display(d)
        r = d[["Line1","Line2","Line3","Line4"]]
        return d,r
    
    def __init__(self,words_df,n,cross,epoch,z):
        Selection_number = int(round(0.85*n))
        best = []
        assert n>1 ,f"Population {n} cannot be less than or equal to 1"
        # assert n>=Selection_number, f"Cannot select {Selection_number} poems from total population of {n}"
        assert cross>1, f"Minimum number for crossover should be 1"
        # assert 2*n >= Selection_number ,f"Cannot make initial population of {n} if only {Selection_number} parents are selected"

        Population_data = self.create_set_of_poems(n)
        for i in range(epoch):
            Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number,best)
            child_population = self.child_poems(selected_polulation_word_array,cross)
            final_population,final_poems = self.selection_from_child_parents(n,child_population,Population_data)
            if i != epoch-1:
                if (i+1)%10 == 0: # User feedback every 10 epochs
                    print("Select best poems")
                    display(final_poems)
                    best = (list(map(int,input("\n Enter best poems numbers: ").strip().split())))[:z]
                Population_data = final_population
                print("Final poems")
                display(final_population)

            else:
                print("###################################################")
                print("Last Poems Standing")
                print("###################################################")
                display(final_population)
                
        

In [14]:
#4 (words_df,intial_population,crossover,epochs,user_selection_poems_number):
poem = poem_generation(words_df,10,2,2,5)


Final poems


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,Gram_Rank,Coh_Rank,Syll_Bonus,Fitness,words_array
0,know mortal pinpoint misses,her trees feel be,word to of burden,turned you figments call,7,4,5,6,17,[8.63054782402313e-06],7.0,4.0,0,0.44,"[know, mortal, pinpoint, misses, her, trees, feel, be, word, to, of, burden, turned, you, figments, call]"
1,are let having knew,road enigma its here,be observed world that,as were pleasing souls,6,7,6,6,11,[0.0],10.0,1.0,0,0.44,"[are, let, having, knew, road, enigma, its, here, be, observed, world, that, as, were, pleasing, souls]"
2,lives and organs hand,forgiveness design of ever,her emptiness me may,fond the in knowledge,6,9,6,6,23,[2.7569805548962773e-05],2.0,6.0,0,0.32,"[lives, and, organs, hand, forgiveness, design, of, ever, her, emptiness, me, may, fond, the, in, knowledge]"
3,finality to causing after,a without i let,i fear clap all,there upon the its,9,5,4,6,15,[3.416258513675822e-05],8.0,7.0,0,0.60,"[finality, to, causing, after, a, without, i, let, i, fear, clap, all, there, upon, the, its]"
4,into years holds is,moved for so how,together we let having,knew are devour me,5,5,7,6,21,[0.0],2.0,1.0,0,0.15,"[into, years, holds, is, moved, for, so, how, together, we, let, having, knew, are, devour, me]"
5,keeping by is remembering,road enigma its here,be observed world that,as were pleasing souls,8,7,6,6,15,[7.705846271449224e-06],6.0,4.0,0,0.50,"[keeping, by, is, remembering, road, enigma, its, here, be, observed, world, that, as, were, pleasing, souls]"
6,call causing after a,without to of know,mortal pinpoint misses her,trees feel be word,6,5,7,4,22,[9.58949758224792e-06],1.0,5.0,0,0.30,"[call, causing, after, a, without, to, of, know, mortal, pinpoint, misses, her, trees, feel, be, word]"
7,of every all we,time call for casts,life and flying in,turning and delight on,6,4,4,6,17,[0.00017415212573475243],4.0,8.0,0,0.60,"[of, every, all, we, time, call, for, casts, life, and, flying, in, turning, and, delight, on]"
8,into appear city upright,dressed alive unique has,heart trees we within,a relevant spirited of,8,9,5,8,17,[0.0],4.0,1.0,0,0.25,"[into, appear, city, upright, dressed, alive, unique, has, heart, trees, we, within, a, relevant, spirited, of]"
9,a a long chest,our never pleasing losers,knell that cold life,eye inside like alive,4,7,4,8,12,[8.434399009840786e-05],8.0,7.0,0,0.75,"[a, a, long, chest, our, never, pleasing, losers, knell, that, cold, life, eye, inside, like, alive]"



###################################################
Last Poems Standing
###################################################


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,Gram_Rank,Coh_Rank,Syll_Bonus,Fitness,words_array
0,know mortal pinpoint misses,her trees feel be,word to of burden,turned you figments call,7,4,5,6,17,[8.63054782402313e-06],7.0,4.0,0,0.44,"[know, mortal, pinpoint, misses, her, trees, feel, be, word, to, of, burden, turned, you, figments, call]"
1,are let having knew,road enigma its here,be observed world that,as were pleasing souls,6,7,6,6,11,[0.0],10.0,1.0,0,0.44,"[are, let, having, knew, road, enigma, its, here, be, observed, world, that, as, were, pleasing, souls]"
2,into years holds is,moved for so how,together we let having,knew are devour me,5,5,7,6,21,[0.0],2.0,1.0,0,0.15,"[into, years, holds, is, moved, for, so, how, together, we, let, having, knew, are, devour, me]"
3,of every all we,time call for casts,life and flying in,turning and delight on,6,4,4,6,17,[0.00017415212573475243],4.0,8.0,0,0.60,"[of, every, all, we, time, call, for, casts, life, and, flying, in, turning, and, delight, on]"
4,souls enigma its here,be observed world that,as were pleasing long,are let having knew,7,6,6,6,12,[0.0],8.0,1.0,0,0.45,"[souls, enigma, its, here, be, observed, world, that, as, were, pleasing, long, are, let, having, knew]"
5,let i fear clap,all there finality to,causing after a without,i be observed world,4,8,7,6,14,[1.5411692542898447e-05],6.0,6.0,0,0.60,"[let, i, fear, clap, all, there, finality, to, causing, after, a, without, i, be, observed, world]"
6,that upon the its,as were pleasing souls,keeping by is remembering,road enigma its here,5,6,8,7,17,[8.990153983357427e-06],3.0,4.0,0,0.35,"[that, upon, the, its, as, were, pleasing, souls, keeping, by, is, remembering, road, enigma, its, here]"
7,every all we time,call for casts life,and flying in turning,and delight on burden,6,4,5,6,15,[0.00015238311001790838],5.0,8.0,0,0.65,"[every, all, we, time, call, for, casts, life, and, flying, in, turning, and, delight, on, burden]"
8,turned you figments call,of know mortal pinpoint,misses her trees feel,be word to of,6,6,5,4,16,[9.58949758224792e-06],4.0,5.0,0,0.45,"[turned, you, figments, call, of, know, mortal, pinpoint, misses, her, trees, feel, be, word, to, of]"
9,unique has heart trees,we within a relevant,spirited of into appear,city upright dressed alive,6,7,8,9,20,[0.0],1.0,1.0,0,0.10,"[unique, has, heart, trees, we, within, a, relevant, spirited, of, into, appear, city, upright, dressed, alive]"


In [11]:
print(list(range(5,8)))

[5, 6, 7]


In [12]:
a = [2,3,4,5]

for i in a:
    print(i)

2
3
4
5
